<a href="https://colab.research.google.com/github/taesangeom/SK-T-Academy/blob/main/Coding_Test/11_%EA%B5%AC%ED%98%84_%EC%A3%BC%EC%B0%A8%EC%9A%94%EA%B8%88_%EC%B9%B4%EC%B9%B4%EC%98%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 초기 내 버전 (카카오 2022 Blind Recruitment LV2)

In [ ]:
# 내가 해결한 버전 (성공)  -> 다만, datetime 등 시간처리는 gpt 참고함 -> 시간처리 잘 공부할 것 , 문제 푸는 시간 너무 많이 걸림
import datetime
import math

def solution(fees, records):
    record_list=[]
    fee_dict={} # 초기에는 번호: 분  -> 나중에 번호: 돈
    num_times={} # 번호:[시간, 시간]

    for record in records:
        time=record.split(" ")[0]
        number=int(record.split(" ")[1])
        if number not in fee_dict:
            fee_dict[number]=0
        if number not in num_times:
            num_times[number]=[]
            num_times[number].append(time)
        else:
            num_times[number].append(time)

    for i in num_times:
        if len(num_times[i])%2==1:  # len으로 하면 원소 수이므로 짝,홀수 주의
            num_times[i].append("23:59")
        for j in range(0,len(num_times[i]),2):
            time_in=datetime.datetime.strptime(num_times[i][j],"%H:%M")
            time_out=datetime.datetime.strptime(num_times[i][j+1],"%H:%M")
            time_diff=time_out-time_in
            diff=time_diff.total_seconds()/60
            fee_dict[i]+=int(diff)

    for i in fee_dict:
        if fee_dict[i]<=fees[0]:
            fee_dict[i]=fees[1]
        else:
            fee_dict[i]=fees[1] + math.ceil((fee_dict[i]-fees[0])/fees[2])*fees[3]

    answer=[money for num,money in sorted(fee_dict.items(),key = lambda x:+x[0])]
    return answer














In [ ]:
solution([180, 5000, 10, 600],["05:34 5961 IN", "06:00 0000 IN", "06:34 0000 OUT", "07:59 5961 OUT", "07:59 0148 IN", "18:59 0000 IN", "19:09 0148 OUT", "22:59 5961 IN", "23:00 5961 OUT"])

[14600, 34400, 5000]

##강사님 버전

In [ ]:
# ==> 코드 정리

def convert_h_to_m( time_HHMM):
    h_part, m_part = time_HHMM.split(":")
    total_min = int(h_part) * 60 + int(m_part) # 산수+위해서 형변형
    return total_min

def solution(fees, records):
    answer = []
    base_time, base_cost, add_time, add_cost = fees
    park_dict = {}
    for record in records: # ["01:23 0000 IN", "03:45 1111 IN",,,]
        park_time, car_id, status = record.split(" ")
        car_id = int(car_id) # key
        park_time, car_id, status = record.split(" ")
        car_record = [convert_h_to_m(park_time), status ] # value
        if car_id in park_dict:
            park_dict[car_id].append( car_record ) # [90,"OUT"]
        else:
            park_dict[car_id] = [car_record]# [83,"IN"]  --> [ [83,IN] ]
    car_dict = {} # 키: 차번호, 값: 요금
    for car, logs in park_dict.items(): # k,v
        out_time_list = [ m for m,sta in logs if sta=="OUT"]
        in_time_list  = [m for m,sta in logs if sta=="IN"]
        if len(logs) % 2 ==1: # 출차 정보가 없는 경우
            out_time_list.append(convert_h_to_m("23:59") )
        total_min = sum(out_time_list) - sum(in_time_list)
        if total_min <= base_time: # fees[0]
            car_dict[car] = base_cost # fees[1]
        else: # 추가 요금이 발생!!!!
            cost = 0
            cost += base_cost
            total_min -= base_time # 누적시간 = 기존누적시간 - 기본 시간
            over_time_base = -( total_min* (-1) // add_time )
            cost += over_time_base * add_cost
            car_dict[car] = cost
    car_cost = dict(sorted(car_dict.items()))
    answer = list(car_cost.values())
    return answer